# Pattern matching test for dictionary representation
$\mathbf A \mathbf s = \mathbf f$

A is dictionary  
s is the sparse representation vector  
f is the full representation

In [2]:
import numpy as np

s_size = 32
f_size = 1024
A = np.random.normal(size = [f_size,s_size])
np.mean(np.var(A,axis=1,ddof=1))

1.000697496636606

$E\{\mathbf A\} = 0$  
$\sigma^2 \{\mathbf A\} = 1$

In [3]:
n_samples = int(10e3)
sparsity = 1e-1
sparsity_int = int(np.ceil(sparsity * s_size))
s = np.zeros([s_size,n_samples])
for col in range(n_samples):
    s[np.random.choice(s_size,size=sparsity_int,replace=False),col] = np.random.normal(loc=0,scale = 1 ,size=sparsity_int)
    # 1 # 2*np.random.rand(sparsity_int) #np.random.normal(loc=0,scale = 1 ,size=sparsity_int)
np.mean(np.var(s,axis=0,ddof=1)*s_size)

4.01932833304363

$E\{\mathbf s\} = 0$  
$\sigma^2 \{\mathbf s\} = \frac{sparsity\_int}{s\_size}$

In [6]:
f = np.dot(A, s)
f_snrs = [10,20,30] #dB
f_noisy = {'snr='+str(snr):f + np.random.normal(scale = sparsity_int * 10**(-snr/10), size = f.shape) for snr in f_snrs}
np.mean(np.var(f,axis=0,ddof=1))

4.023934873361924

$E\{\mathbf f\} =E\{\mathbf A \mathbf s\}=E\{\mathbf A\}E\{\mathbf s\}= 0$  
$\sigma^2 \{\mathbf f\ | \mathbf A\} = E\{\mathbf A \mathbf s \mathbf s^H \mathbf A^H\} = \mathbf A E\{\mathbf s \mathbf s^H \}\mathbf A^H = \mathbf A \sigma^2 \{s\} \mathbf A^H$

In [10]:
from omp import OMP
shat = {}
for key in f_noisy.keys():
    shat_part = np.zeros([s_size,n_samples])
    for col in range(n_samples):
        shat_part[:,col] = OMP(A, f_noisy[key][:,col],sparsity_int)[0]
    shat[key] = shat_part
    print(key+', error = ' + str(np.linalg.norm(s-shat_part)))

snr=20, error = 0.25251907147674146
snr=10, error = 2.709268496204634
snr=30, error = 0.025107106387377495
